In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow_datasets as tfds

ontology_and_seq_fp = "../data/intermediary/drosophila_protein_ontology_and_seqs.csv"

# preprocess data so its more vector-y for tensorflow ingest

df = pd.read_csv(ontology_and_seq_fp)
relevant_subset = df[df.qualifier.isin(["enables", "involved_in"])].dropna()
interesting_go_names = [name for (name, freq) in relevant_subset.go_name.value_counts().to_dict().items() if 1 < freq]  # <- probably need to change the filter step !!
df = df[df.go_name.isin(interesting_go_names)]
one_row_per_gene = pd.DataFrame(index=df.seq.unique(), columns=interesting_go_names).fillna(0)
for _, row in df.iterrows():
    one_row_per_gene.loc[row.seq, row.go_name] = 1
one_row_per_gene = one_row_per_gene.reset_index().rename(columns={"index": "seq"})
# seq_len_max = one_row_per_gene.seq.str.len().max()
# one_row_per_gene.seq = one_row_per_gene.seq.str.pad(seq_len_max, side="right", fillchar=" ")
one_row_per_gene.head()

,seq,asymmetric neuroblast division,protein import into nucleus,nuclear-transcribed mRNA poly(A) tail shortening,establishment or maintenance of neuroblast polarity,methionine biosynthetic process,mRNA export from nucleus,establishment of imaginal disc-derived wing hair orientation,ribosome biogenesis,sarcomere organization,...,endoplasmic reticulum unfolded protein response,olfactory learning,neuropeptide receptor activity,positive regulation of axon guidance,neuron projection development,wing disc dorsal/ventral pattern formation,adult locomotory behavior,synaptic growth at neuromuscular junction,anterograde axonal transport,sodium ion transmembrane transport
0,MDCFKVFEVVFQSEINPLLLIPAVATIALTLCCYCYHGYQWIRDRR...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,MFTTRKEVDAHVHKMLGKLQPGRERDIKGLAVARLYMKVQEYPKAI...,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,MEPHFFFTVLWMLLMGTSSTYAQEIFGYCRTPDENSGTCINLRECG...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,MSNVHQFDTQTMAESPQIRRDMGRLCATWPSKDSEDGAGTALRAAT...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,MALVYGVEKKTVPTHMKFVMGGTSGMLATCIVQPLDLLKTRMQISG...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
vocab = set()
for seq in one_row_per_gene.seq:
    vocab.update(seq)
encoder = {char: i for i, char in enumerate(vocab)}
def encode(seq):
    return np.array([encoder[residue] for residue in seq])
X = one_row_per_gene.pop("seq") #.apply(encode)
y = one_row_per_gene.values

In [26]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.25, random_state=42)

In [27]:
examples_train = tf.data.Dataset.from_tensor_slices((X_train, y_train)) 
examples_test = tf.data.Dataset.from_tensor_slices((X_test, y_test)) 

In [ ]:
def encode(text_tensor, label):
    encoded_text = encoder.encode(text_tensor.numpy())
    return encoded_text, label

In [28]:
examples_test

<TensorSliceDataset shapes: ((), (144,)), types: (tf.string, tf.int64)>

In [5]:
BUFFER_SIZE = 10000
BATCH_SIZE = 16

In [15]:
train_dataset = (examples_train
                 .map
                 .shuffle(BUFFER_SIZE)
                 .padded_batch(BATCH_SIZE, padded_shapes=([""],"",)))
                

test_dataset = (examples_test
                .padded_batch(BATCH_SIZE,  padded_shapes=([[]]))
               )

TypeError: Padded shape [''] must be a 1-D tensor of tf.int64 values, but its element type was string.